In [ ]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 42.0 MB/s eta 0:00:28tcmalloc: large alloc 1147494400 bytes == 0x5619e526c000 @  0x7fd0f67a7615 0x5619ab51b4cc 0x5619ab5fb47a 0x5619ab51e2ed 0x5619ab60fe1d 0x5619ab591e99 0x5619ab58c9ee 0x5619ab51fbda 0x5619ab591d00 0x5619ab58c9ee 0x5619ab51fbda 0x5619ab58e737 0x5619ab610c66 0x5619ab58ddaf 0x5619ab610c66 0x5619ab58ddaf 0x5619ab610c66 0x5619ab58ddaf 0x5619ab520039 0x5619ab563409 0x5619ab51ec52 0x5619ab591c25 0x5619ab58c9ee 0x5619ab51fbda 0x5619ab58e737 0x5619ab58c9ee 0x5619ab51fbda 0x5619ab58d915 0x5619ab51fafa 0x5619ab58dc0d 0x5619ab58c9ee
     |█████████████████               | 1055.7 MB 20.7 MB/s eta 0:00:45tcmalloc: large alloc 1434370048 bytes == 0x561a298c2000 @  0x7fd0f67a7615 0x5619ab51b4cc 0x5619ab5fb47a 0x5619ab51e2ed 0x5619ab60fe1d 0x5619ab591e99 0x5619ab58c9ee 0x5619ab51fbda 0x5619ab591d00 0x5619ab58c9ee 0x5619ab51fbda 0x5619ab58e737 0x5619ab610c66 0x5619ab58

In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
#BERT-Tokenization/collecting tokenizers

#BERT-Tokenization/collecting tokenizers

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
#Testing with own sentences/reviews

#Testing with own sentences/reviews

In [5]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [18]:
tokens = tokenizer.encode('It was worst service', return_tensors='pt')

In [22]:
tokens = tokenizer.encode('It was great service', return_tensors='pt')

In [43]:
tokens = tokenizer.encode('It was disgusting service', return_tensors='pt')

In [44]:
result = model(tokens)

In [45]:
#Probability scores as the output/doing argmax to pick the most higher probability

#Probability scores as the output/doing argmax to pick the most higher probability

In [46]:
result.logits

tensor([[ 3.0481,  1.3916,  0.1251, -1.7785, -2.0451]],
       grad_fn=<AddmmBackward>)

In [47]:
#Sentiment Score
#lower the score : negative review
#Higher the score : positive review

#Sentiment Score
#lower the score : negative review
#Higher the score : positive review

In [48]:
int(torch.argmax(result.logits))+1

1

In [ ]:
#Scraping/collecting reviews from the website

In [33]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [34]:
import numpy as np
import pandas as pd

In [35]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [36]:
df

,review
0,The food is fresh and tasty. The scallop cevi...
1,Don't come here expecting legit Mexican food b...
2,Out of all the restaurants that I tried in Syd...
3,I was pleasantly surprised at what a great job...
4,Really nice (upmarket) Mexican restaurant. Goo...
5,We came here on a Thursday night @ 5pm and by ...
6,Ordered feed me for $59 along with that.. Food...
7,"The best tasting cauliflower taco, mushroom ce..."
8,Have been here twice and have absolutely loved...
9,Why is it that all Mexican food outside of Mex...


In [37]:
df['review'].iloc[1]

"Don't come here expecting legit Mexican food but a modern twist on some staples. Loud party area, fun drinks and friendly staff make this a hip meeting area for large groups. Drinks were better than the food. They stuff the families toward the back but lack any amenities (no changing table) except a high chair. Service started off friendly but it took a while to get someone to take our order and then they forgot our dish which came out cold when we asked for it. Then we had to flag someone down to pay the bill. The watermelon salad was tasty but not complex, tossed with a few cucumbers and pistachios. The corn lollipops with spicy mayo sauce were probably the best dish. The beef empanadas were cold and average though the salsa was an interesting pickled onion. Definitely skip the octopus dish- it was dry, over cooked and lacked flavor though the presentation looked promising. It came with squares of polenta and dallops of potato mash with a dramatic squid ink streak across the dish bu

In [38]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [39]:
sentiment_score(df['review'].iloc[1])

3

In [ ]:
# having review and sentiment score

#having reviews and sentiment scores using dataframe

In [40]:

df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
#Sentiment Score
#lower the score : negative review
#Higher the score : positive review

#Sentiment Score
#lower the score : negative review
#Higher the score : positive review

In [41]:
df


,review,sentiment
0,The food is fresh and tasty. The scallop cevi...,4
1,Don't come here expecting legit Mexican food b...,3
2,Out of all the restaurants that I tried in Syd...,5
3,I was pleasantly surprised at what a great job...,5
4,Really nice (upmarket) Mexican restaurant. Goo...,4
5,We came here on a Thursday night @ 5pm and by ...,4
6,Ordered feed me for $59 along with that.. Food...,2
7,"The best tasting cauliflower taco, mushroom ce...",5
8,Have been here twice and have absolutely loved...,5
9,Why is it that all Mexican food outside of Mex...,3


In [ ]:
#Manual checking using dataframe

In [42]:
df['review'].iloc[3]

"I was pleasantly surprised at what a great job they did with Mexican food at this downtown Sydney eatery. Very friendly staff, a substantial bar area, and yummy cocktails round out this dining experience in a superb way. Try the watermelon salad, and ANY of the tacos. You won't be disappointed!"